# Poisson Blending
Contexto breve: Recordemos la primera vez que intentamos mezclar 2 imagenes que no tienen relación alguna, es probable que el resultado no fuera muy agradable, esto se debe a que las intensidades entre cada imagen puede que sean muy distintas entre si, creando un cambio bruzco entre la imagen de fondo y la que esta encima de.

Existe un método para "mezclar" una imagen fuente dentro de una imagen objetivo, conocido como Poisson Blending, aunque también es conocido como Gradient Domain Image Blending (Mezcla de imágenes en el dominio del gradiente) debido al uso de este operador.

# Explicación simplificada para Poisson Blending
El método es relativamente sencillo, sin embargo, las justificaciones mátematicas pueden llegar asustar un poco si se tiene poca experiencia en cálculo multidimensional. Por lo tanto, harémos una explicación un poco más simplificada para explicar el comportamiento general del algoritmo, enfocandonos simplemente en un vector de intensidades (una fila de pixeles, en lugar de una imagen de 2 dimensiones).

## Modelado y definiciones importantes
Antes de entrar de lleno al algoritmo, es importante hablar un poco de las bases de este modelo. Algunas palabras que usaremos son:
- Imagen objetivo: Representa la imagen que se encontrará en el fondo
- Imagen fuente: Representa la imagen que se colocará sobre la imagen fuente
- Intensidad: Cantidad de que tanto brilla un pixel (si solo esta en escala de grises)
### ¿Qué es el gradiente de una imagen?
El calculo del gradiente es muy importante, por lo que veremos como es que este se puede calcular en el caso de 1 dimensión.

Supongamos que tenemos nuestra imagen en escala de grises:

<img src="./img1.png" alt="imagen_objetivo" width="50%" height="auto"/>

Cada pixel tiene su propio valor de intensidad, de la imagen notamos que tenemos 8 pixeles, los cuales contienen los valores de intensidad $I_1,...,I_8$. Para una mejor visualización de esto, podemos crear la siguiente gráfica discretizada:

<img src="./hist1.png" alt="hist1" width="50%" height="auto"/>

Además, cada pixel tiene su propio valor de gradiente.

Nota:Cuando hablamos de gradiente en 1 dimensión, sabemos que esto se reduce al calculo de la derivada. 

Supongamos que queremos calcular el gradiente del pixel 3, entonces tenemos: 

$$
\frac{I_{3+\Delta p}-I_3}{\Delta p}
$$

Nuestro $\Delta p$ más pequeño es $1$ debido a la distancia entre los pixeles, esto nos deja:

$$
\frac{I_{3+ 1}-I_3}{1}= I_4-I_3=5-3=2
$$

Definimos el vector gradiente entre los pixeles <p,q> como:
$$
v_{p,q}=I_p-I_q = -v_{q,p}
$$


Vemos que el gradiente v_{p,q}, no es más que el cambio entre las intensidades desde q hasta p. Visualmente podemos ver esto como:

<img src="./gradiente.png" alt="gradiente" width="50%" height="auto"/>

### Un pequeño problema
Ahora que sabemos como describir un pixel en terminos de intensidades y gradientes, podemos ver una aplicación sencilla.

Supongamos que por alguna razón, los pixeles 3, 4, 5 y 6 desaparecieron y no sabemos que intensidades tenian. Sin embargo, tenemos guardados los valores de los gradientes que le corresponden a estos 4 pixeles. Además, las intensidades de los pixeles 2 y 7 son conocidas, sabiendo esto, ¿Cuál es el valor de intensidad de los pixeles desaparecidos?

<img src="./hist3.png" alt="gradiente" width="50%" height="auto"/>

Si recordamos la definición del gradiente:
$$
\begin{align}
v_{p,q}&=I_p-I_q\\
I_p &= v_{p,q} + I_q
\end{align}
$$

Sustituyendo tenemos:
$$
\begin{align}
I_3 &= v_{3,2} + I_2\\
I_3 &= -1 + 4\\
I_3 &= 3
\end{align}
$$

Ahora, que conocemos $I_3$, es fácil notar que siguiendo el mismo procedimiento, podemos encontrar $I_4$, y los demás pixeles, consiguiendo reconstruir la imagen original. 
Esto fue gracias a 2 cosas, conociamos los gradientes de los pixeles desconocidos y conociamos los valores de frontera o limites que estaban a la izquierda y derecha de nuestros pixeles perdidos.

Visto de otra forma, para nosotros poder reconstruir una imagen, ocupamos encontrar los valores de intensidad $I_p$, $I_q$, cuya diferencia entre ellos sea minima, respecto a la diferencia de ellos a su gradiente, $(I_p - I_q - v_{p,q})^2$, usamos el cuadrado ya que lo unico que nos importa es que la cantidad sea pequeña. Si la diferencia entre las intensidades, es muy parecida al gradiente entre los pixeles, entonces la imagen será muy similar a la imagen original

En general, una imagen puede ser reconstruida si minimizamos el valor de la función de energía $h$:

$$ 
h(I_u,...,I_k)= (I_u - I_{(u-1)} - v_{u,u-1})^2 + (I_{(u-1)} - I_u - v_{u-1,u})^2\quad +\quad ...\quad +\quad (I_k - I_{(k-1)} - v_{k,k-1})^2 + (I_{(k-1)} - I_k - v_{k-1,k})^2
$$

$$ 
h = \sum_{j=u}^{k} (I_j - I_{(j-1)} - v_{j,j-1})^2 + (I_{(j-1)} - I_j - v_{j-1,j})^2,\quad u,...,k = \text{pixeles a reconstruir}
$$


La función de energia $h$ nos mide que tanto se parecen los gradientes originales con los gradientes nuevos, minimizando esta función conseguimos reconstruir nuestra imagen.

Para nuestro caso particular, la función de energía sería:

$$
\begin{align}
h(I_3,I_4,I_5,I_6)&= (I_3 - I_{2} - v_{3,2})^2 + (I_{2} - I_3 - v_{2,3})^2\\
&+(I_4 - I_{3} - v_{4,3})^2 + (I_{3} - I_4 - v_{3,4})^2\\
&+(I_5 - I_{4} - v_{5,4})^2 + (I_{4} - I_5 - v_{4,5})^2\\
&+(I_6 - I_{5} - v_{6,5})^2 + (I_{5} - I_6 - v_{5,6})^2\\
&+(I_7 - I_{6} - v_{7,6})^2 + (I_{6} - I_7 - v_{6,7})^2
\end{align}
$$

M

Ocupamos minimizar esta función, como simplemente es una función cuadratica, igualamos su derivada a cero respecto a cada pixel a reconstruir, viendo un caso particular, usamos un caso dónde no nos encontramos en la frontera, por ejemplo $I_5$. También como sabemos $v_{p,q} = -v_{q,p}$, tenemos:
$$
\begin{align}
\frac{\partial{h}}{\partial{I_5}} &= 0\\
2(I_5−I_4−v_{5,4})−2(I_4−I_5−v_{4,5})−2(I_6−I_5−v_{6,5})+2(I_5−I_6−v_{5,6}) &= 0\\
I_5−I_4−v_{5,4}−I_4+I_5+v_{4,5}−I_6+I_5+v_{6,5}+I_5−I_6−v_{5,6} &= 0\\
4I_5−2I_6−2I_4−v_{5,4}+v_{4,5}+v_{6,5}−v_{5,6} &= 0\\
4I_5−2I_6−2I_4−2v_{5,4}−2v_{5,6} &= 0\\
2I_5−I_6−I_4−v_{5,4}−v_{5,6} &= 0\\
2I_5−I_6−I_4 &= v_{5,4}+v_{5,6} =-3\\
2I_5−I_6−I_4 &=-3
\end{align}
$$

Tenemos que nuestros valores conocidos, a la derecha de la igualdad, son simplemente nuestros gradientes, mientras que a la izquierda solo tenemos valores desconocidos.

Para el caso donde el pixel a reconstruir es frontera, por ejemplo $I_3$ tenemos:
$$
\begin{align}
\frac{\partial{h}}{\partial{I_3}} &= 0\\
2(I_3−I_2−v_{3,2})−2(I_2−I_3−v_{2,3})−2(I_4−I_3−v_{4,3})+2(I_3−I_4−v_{3,4}) &= 0\\
4I_3−2I_4−2I_2−2v_{3,2}−2v_{3,4} &= 0\\
2I_3−I_4−I_2−v_{3,2}−v_{3,4} &= 0\\
2I_3−I_4−I_2 &= v_{3,2}+v_{3,4}\\
2I_3−I_4 &= v_{3,2}+v_{3,4} + I_2 = 1\\
2I_3−I_4 &=1
\end{align}
$$

Notamos que llegamos a una expresión similar, sin embargo, al ser un pixel de frontero, $I_2$, pertenece a nuestras condiciones de frontera, por lo que la ecuación se simplifica.

Aplicando las demas derivadas, llegamos al siguiente sistema de ecuaciones lineales:

$$
\begin{align}
2I_3−I_4 &=1\\
2I_4−I_3−I_5 &=3\\
2I_5−I_6−I_4 &=-3\\
2I_6−I_5 &=8
\end{align}
$$

Resolviendo este sistema por cualquier método deseado, obtenemos nuestros valores de intensidad perdidos.

## Aplicación de lo visto para la mezcla de imagenes
Hasta el momento, solo hemos hablado sobre como calcular el gradiente de los pixeles, y un pequeño problema donde se reconstruyeron las intensidades de una imagen, en la cual solo se conocian sus gradientes originales.  
Sin embargo, estos 2 aspectos vistos, son suficientes para mezclar 2 imagenes.

Supongamos que tenemos una imagen fuente (esta será la imagen que queremos recortar y pegar en algun destino), la cual tiene ciertos valores de intensidad, como vemos en la siguiente imagen: 
 

In [ ]:
import numpy as np

In [1]:
# idk

def PoissonBlending(src_img, mask, src_grad, trgt_img, x0,y0, bc):

    for mask_row in mask:
        for mask_value in mask_row
            if value == 1:
                
                # do something
            else: 
                continue

def Energy
        
    